In [1]:
#### importing original data ####
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import seaborn as sns 
import re 
%matplotlib inline
import glob

from tqdm import tqdm


In [2]:
# take one
# original_data = pd.read_csv('/scratch/gh8sj/brown_nsclc/processed_nsclc_data_1_10_24.csv', index_col=0)
# take 2
original_data = pd.read_csv('/scratch/gh8sj/brown_nsclc/data/processed_data/tumor_stroma_processed_nsclc_data_4_1_24.csv', index_col=0)


#### Deal with the non-numerics that might come up when converting back and forth between CSV ####
original_data['XCenter'] = pd.to_numeric(original_data['XCenter'], errors='coerce')
original_data['YCenter'] = pd.to_numeric(original_data['YCenter'], errors='coerce')
original_data['XMin'] = pd.to_numeric(original_data['XMin'], errors='coerce')
original_data['XMax'] = pd.to_numeric(original_data['XMax'], errors='coerce')
original_data['YMin'] = pd.to_numeric(original_data['YMin'], errors='coerce')
original_data['YMax'] = pd.to_numeric(original_data['YMax'], errors='coerce')

original_data = original_data[original_data['XCenter'].apply(lambda x: isinstance(x, (int, float)) or (isinstance(x, str) and x.isnumeric()))]
print(len(original_data))


#### Change all to float; needed for this code to run 

original_data['XCenter'] = original_data['XCenter'].astype(float)
original_data['YCenter'] = original_data['YCenter'].astype(float)
original_data['XMin'] = original_data['XMin'].astype(float)
original_data['XMax'] = original_data['XMax'].astype(float)
original_data['YMin'] = original_data['YMin'].astype(float)
original_data['YMax'] = original_data['YMax'].astype(float)



5307540


In [3]:
group_names = original_data.groupby("ImageLocation").groups
grouped = original_data.groupby("ImageLocation")
slide_names = list(group_names.keys())
print(len(slide_names))


414


In [4]:
list(original_data.columns)

['ImageLocation',
 'XMin',
 'XMax',
 'YMin',
 'YMax',
 'MHCI',
 'CD56',
 'CD8',
 'CD3',
 'IFNy',
 'IFNy (Opal 620) Cytoplasm Intensity',
 'PanCyto',
 'ClassifierLabel',
 'ROI',
 'PatientID',
 'XCenter',
 'YCenter',
 'CellArea',
 'Cytoplasm Area (µm²)',
 'Membrane Area (µm²)',
 'Nucleus Area (µm²)',
 'Nucleus Perimeter (µm)',
 'Nucleus Roundness',
 'CD56+PanCyto-',
 'CD56+CD3-',
 'PanCyto+CD3-',
 'PanCyto+MHCI+',
 'PanCyto+MHCI-',
 'PanCyto-CD56-',
 'CD3+CD8-',
 'CD3+CD8+',
 'CD3+CD8+IFNy+',
 'CD3+CD8+IFNy-',
 'CD3+CD8-IFNy+',
 'CD3+CD8-IFNy-',
 'CD56+CD3-IFNy+',
 'CD56+CD3-IFNy-',
 'CD3+CD8+IFNy+ Tumor',
 'CD3+CD8+IFNy+ Stroma',
 'CD3+CD8+IFNy+ Glass',
 'CD3+CD8+IFNy- Tumor',
 'CD3+CD8+IFNy- Stroma',
 'CD3+CD8+IFNy- Glass',
 'CD3+CD8-IFNy+ Tumor',
 'CD3+CD8-IFNy+ Stroma',
 'CD3+CD8-IFNy+ Glass',
 'CD3+CD8-IFNy- Tumor',
 'CD3+CD8-IFNy- Stroma',
 'CD3+CD8-IFNy- Glass',
 'CD56+CD3-IFNy+ Tumor',
 'CD56+CD3-IFNy+ Stroma',
 'CD56+CD3-IFNy+ Glass',
 'CD56+CD3-IFNy- Tumor',
 'CD56+CD3-IFNy- St

In [7]:
header = original_data.head()
header.to_csv('/scratch/gh8sj/brown_nsclc/mycellTypes.csv')
# I think I want to use all stains like a barcode instead of classifying as a cell type, at least at first 
cell_columns = []
for i in original_data.columns:
    if '+' in i:
        cell_columns.append(i)
#cell_types = ['CD8-CD3+IFNy+','CD8-CD3+IFNy-','CD8+CD3+IFNy+','CD8+CD3+IFNy-','CD56+CD3-IFNy+','CD56+CD3-IFNy-','PanCyto+MHCI+','PanCyto+MHCI-']
cells_to_remove = ['CD56+CD3-','CD56+CD3+','CD3+CD8-','CD3+CD8+']
cell_types = [item for item in cell_columns if item not in cells_to_remove]
cell_types

['CD56+PanCyto-',
 'PanCyto+CD3-',
 'PanCyto+MHCI+',
 'PanCyto+MHCI-',
 'CD3+CD8+IFNy+',
 'CD3+CD8+IFNy-',
 'CD3+CD8-IFNy+',
 'CD3+CD8-IFNy-',
 'CD56+CD3-IFNy+',
 'CD56+CD3-IFNy-',
 'CD3+CD8+IFNy+ Tumor',
 'CD3+CD8+IFNy+ Stroma',
 'CD3+CD8+IFNy+ Glass',
 'CD3+CD8+IFNy- Tumor',
 'CD3+CD8+IFNy- Stroma',
 'CD3+CD8+IFNy- Glass',
 'CD3+CD8-IFNy+ Tumor',
 'CD3+CD8-IFNy+ Stroma',
 'CD3+CD8-IFNy+ Glass',
 'CD3+CD8-IFNy- Tumor',
 'CD3+CD8-IFNy- Stroma',
 'CD3+CD8-IFNy- Glass',
 'CD56+CD3-IFNy+ Tumor',
 'CD56+CD3-IFNy+ Stroma',
 'CD56+CD3-IFNy+ Glass',
 'CD56+CD3-IFNy- Tumor',
 'CD56+CD3-IFNy- Stroma',
 'CD56+CD3-IFNy- Glass',
 'PanCyto+MHCI+ Tumor',
 'PanCyto+MHCI+ Stroma',
 'PanCyto+MHCI+ Glass',
 'PanCyto+MHCI- Tumor',
 'PanCyto+MHCI- Stroma',
 'PanCyto+MHCI- Glass']

In [8]:
header = original_data.head()
header.to_csv('/scratch/gh8sj/brown_nsclc/mycellTypes.csv')
# I think I want to use all stains like a barcode instead of classifying as a cell type, at least at first 
cell_columns = []
for i in original_data.columns:
    if '+' in i:
        cell_columns.append(i)
#cell_types = ['CD8-CD3+IFNy+','CD8-CD3+IFNy-','CD8+CD3+IFNy+','CD8+CD3+IFNy-','CD56+CD3-IFNy+','CD56+CD3-IFNy-','PanCyto+MHCI+','PanCyto+MHCI-']
cells_to_remove = ['CD56+CD3-','CD56+CD3+','CD3+CD8-','CD3+CD8+','CD56+PanCyto-','PanCyto+CD3-','PanCyto+MHCI+','PanCyto+MHCI-','CD3+CD8+IFNy+','CD3+CD8+IFNy-','CD3+CD8-IFNy+','CD3+CD8-IFNy-','CD56+CD3-IFNy+','CD56+CD3-IFNy-']
cell_types = [item for item in cell_columns if item not in cells_to_remove]
cell_types

cell_neighbors = []
for i in range(len(cell_types)):
                    cell_neighbors.append(cell_types[i] + ' Neighbors')
zeros_df = np.zeros(( len(original_data),len(cell_neighbors)))
print("len zeros df"+str(len(zeros_df)))
zeros_df = pd.DataFrame(zeros_df, columns = cell_neighbors)
print(len(zeros_df))
print(len(original_data))
original_data = original_data.reset_index()
concat_data = pd.concat([original_data, zeros_df], axis=1)
df=concat_data

len zeros df5307540
5307540
5307540


In [9]:
for column_name in cell_types:
    original_data[column_name] = pd.to_numeric(original_data[column_name], errors='coerce')
    

In [10]:
df.columns

Index(['index', 'ImageLocation', 'XMin', 'XMax', 'YMin', 'YMax', 'MHCI',
       'CD56', 'CD8', 'CD3', 'IFNy', 'IFNy (Opal 620) Cytoplasm Intensity',
       'PanCyto', 'ClassifierLabel', 'ROI', 'PatientID', 'XCenter', 'YCenter',
       'CellArea', 'Cytoplasm Area (µm²)', 'Membrane Area (µm²)',
       'Nucleus Area (µm²)', 'Nucleus Perimeter (µm)', 'Nucleus Roundness',
       'CD56+PanCyto-', 'CD56+CD3-', 'PanCyto+CD3-', 'PanCyto+MHCI+',
       'PanCyto+MHCI-', 'PanCyto-CD56-', 'CD3+CD8-', 'CD3+CD8+',
       'CD3+CD8+IFNy+', 'CD3+CD8+IFNy-', 'CD3+CD8-IFNy+', 'CD3+CD8-IFNy-',
       'CD56+CD3-IFNy+', 'CD56+CD3-IFNy-', 'CD3+CD8+IFNy+ Tumor',
       'CD3+CD8+IFNy+ Stroma', 'CD3+CD8+IFNy+ Glass', 'CD3+CD8+IFNy- Tumor',
       'CD3+CD8+IFNy- Stroma', 'CD3+CD8+IFNy- Glass', 'CD3+CD8-IFNy+ Tumor',
       'CD3+CD8-IFNy+ Stroma', 'CD3+CD8-IFNy+ Glass', 'CD3+CD8-IFNy- Tumor',
       'CD3+CD8-IFNy- Stroma', 'CD3+CD8-IFNy- Glass', 'CD56+CD3-IFNy+ Tumor',
       'CD56+CD3-IFNy+ Stroma', 'CD56+CD3-IFN

## 30 um Neighborhood

In [52]:
def apply_to_group(df, cell_types,cell_neighbors): #xbound = 100 ybound = 100, radius must be manually entered
    """ Applying func to each row. Meaning for each row, every row on the slide is used/iterated over and passed through the functiions below. 
    This is the base function that can do an entire single slide, any function that iterates over multiple slides or compares groups must go 
    through this function last. 
    """

    def func(row): 
        ''' This function, for a single row will iterate over the entire slide and check for whether the difference between
        the XCenter and YCenter of each cell is within a "neighbor range" for that particular cell, and append its index to 
        a list for later use at gathering information about that cell from the main dataframe.'''
        row_prime = row.filter(items = ['XCenter','YCenter'])
        diff = group_prime.drop(row_prime.name).subtract(row_prime.transpose())
        diff = diff.abs().apply(pd.to_numeric, errors='coerce')
        threshold = 30**2  # Set your threshold value
        neighbors = diff.query('(XCenter**2 + YCenter**2) < @threshold') # could be changed to diff<30 or diff< 100 and so on 
        neighbor_count = list(neighbors.index) # make a list of indexes of each column that fits 'neighbor' criteria 
        return(list(neighbor_count))
    
    
    def func_2(row,cell_types,cell_neighbors):
        ''' Takes the original data function and uses information of neighbor indices and provides rest of information about the 
        neighbors of each cell and adds it to the dataframe on the end in Neighbor Columns'''
        neighbor_info = pd.DataFrame(group, index = neighbor_count[row['Slide Index']])# make a dataframe with only rows included in the index , use slide index because the list used is per group
        neighbor_sum = neighbor_info.sum(axis = 0) # sum all columns 
        #print(neighbor_sum)
        row.loc[cell_neighbors] = neighbor_sum.loc[cell_types].to_numpy() # add each values to the 
        return row
    
    



    #print(len(df)) # also helps to note where we are when iterating through multiple slides and what the estimate time will be 
    df.insert(0, 'Slide Index', range(0, len(df))) # THIS IS LIKE A SWTICH THAT MUST BE TURNED ON AND OFF, YOU CANT RUN SOMETHING NEW MORE THAN ONCE 
    group_prime = df.filter(items = ['XCenter','YCenter'])
    group = df
    neighbor_count = group_prime.apply(lambda x: func(x), axis = 1) # apply the inner function to every row, obtain the index
    neighbor_count = list(neighbor_count)
    neighbor_info =  group.apply(lambda row: func_2(row,cell_types,cell_neighbors), axis = 1) # use the list created (now as variable neighbor_count, in the inner function) 
#     print(neighbor_info)
    return neighbor_info


def single_slide_neighbors(df, slide_number= 0): # this entire function could really be replaced by having a column that is "image number" and just numbering them 
    # down the list based on what the image location is, then this is an extra step that can be done earlier 
    ''' This function is meant to take a integer as an input and index from a created list to a slide within the dataframe 
    This is what is then fed to apply_to_group to feed it only a single slidw without having to use the actual image location to call it '''

    group_names = df.groupby("ImageLocation").groups
    grouped = df.groupby("ImageLocation")
    slide_names = list(group_names.keys())
    #print(len(slide_names))
    single_slide = grouped.get_group(slide_names[slide_number])
    print(single_slide)

    return single_slide
    




In [ ]:
# the first run got to 88% (427/487) before my rivanna allocation ran out its here: "/scratch/gh8sj/brown_nsclc/nsclc_30_rad_test.csv"
# second run seems to have been successful

In [69]:
test_df = pd.read_csv("/scratch/gh8sj/brown_nsclc/data/neighbor_data/tumor_stroma/tumor_stroma_nsclc_30_rad.csv")

In [70]:
len(test_df)

5307540

## 200 um neighborhood

## go back and load the data again and run the code until you reach this header for 30 um, then skip and come to here.

In [2]:
def apply_to_group(df, cell_types,cell_neighbors): #xbound = 100 ybound = 100, radius must be manually entered
    """ Applying func to each row. Meaning for each row, every row on the slide is used/iterated over and passed through the functiions below. 
    This is the base function that can do an entire single slide, any function that iterates over multiple slides or compares groups must go 
    through this function last. 
    """

    def func(row): 
        ''' This function, for a single row will iterate over the entire slide and check for whether the difference between
        the XCenter and YCenter of each cell is within a "neighbor range" for that particular cell, and append its index to 
        a list for later use at gathering information about that cell from the main dataframe.'''
        row_prime = row.filter(items = ['XCenter','YCenter'])
        diff = group_prime.drop(row_prime.name).subtract(row_prime.transpose())
        diff = diff.abs().apply(pd.to_numeric, errors='coerce')
        threshold = 200**2  # Set your threshold value
        neighbors = diff.query('(XCenter**2 + YCenter**2) < @threshold') # could be changed to diff<30 or diff< 100 and so on 
        neighbor_count = list(neighbors.index) # make a list of indexes of each column that fits 'neighbor' criteria 
        return(list(neighbor_count))
    
    
    def func_2(row,cell_types,cell_neighbors):
        ''' Takes the original data function and uses information of neighbor indices and provides rest of information about the 
        neighbors of each cell and adds it to the dataframe on the end in Neighbor Columns'''
        neighbor_info = pd.DataFrame(group, index = neighbor_count[row['Slide Index']])# make a dataframe with only rows included in the index , use slide index because the list used is per group
        neighbor_sum = neighbor_info.sum(axis = 0) # sum all columns 
        #print(neighbor_sum)
        row.loc[cell_neighbors] = neighbor_sum.loc[cell_types].to_numpy() # add each values to the 
        return row
    
    



    #print(len(df)) # also helps to note where we are when iterating through multiple slides and what the estimate time will be 
    df.insert(0, 'Slide Index', range(0, len(df))) # THIS IS LIKE A SWTICH THAT MUST BE TURNED ON AND OFF, YOU CANT RUN SOMETHING NEW MORE THAN ONCE 
    group_prime = df.filter(items = ['XCenter','YCenter'])
    group = df
    neighbor_count = group_prime.apply(lambda x: func(x), axis = 1) # apply the inner function to every row, obtain the index
    neighbor_count = list(neighbor_count)
    neighbor_info =  group.apply(lambda row: func_2(row,cell_types,cell_neighbors), axis = 1) # use the list created (now as variable neighbor_count, in the inner function) 
#     print(neighbor_info)
    return neighbor_info


def single_slide_neighbors(df, slide_number= 0): # this entire function could really be replaced by having a column that is "image number" and just numbering them 
    # down the list based on what the image location is, then this is an extra step that can be done earlier 
    ''' This function is meant to take a integer as an input and index from a created list to a slide within the dataframe 
    This is what is then fed to apply_to_group to feed it only a single slidw without having to use the actual image location to call it '''

    group_names = df.groupby("ImageLocation").groups
    grouped = df.groupby("ImageLocation")
    slide_names = list(group_names.keys())
    #print(len(slide_names))
    single_slide = grouped.get_group(slide_names[slide_number])
    print(single_slide)

    return single_slide
    




In [ ]:
def create_neighbor_csv(cell_types,cell_neighbors):
    ''' This uses the inner functions that create columns of neighbor cells and exports as a new csv that can be later imported 
    and code can be performed for data summary.'''
    for i in tqdm(list(range(0,414))): # THIS RANGE MUST BE ADJUSTED IN TERMS OF HOW MANY SLIDES THERE ARE , OR UNIQUE IMAGE LOCATIONS
        slide_number = i
        def single_slide_neighbors(df,slide_number = 1,cell_types = cell_types,cell_neighbors = cell_types):
            ''' This function is meant to take a integer as an input and index from a created list to a slide within the dataframe 
            This is what is then fed to apply_to_group to feed it only a single slidw without having to use the actual image location to call it '''
            group_names = df.groupby("ImageLocation").groups
            grouped = df.groupby("ImageLocation")
            slide_names = list(group_names.keys())
            single_slide = grouped.get_group(slide_names[slide_number])
            return single_slide

        slide_info = single_slide_neighbors(df, slide_number)
        slide_stats = apply_to_group(slide_info,cell_types,cell_neighbors)
        #print(slide_stats)
        # slide_stats.to_csv(r"C:\OyinProject\Proj1.csv", index = False)

        with open(r"/scratch/gh8sj/brown_nsclc/data/neighbor_data/tumor_stroma/tumor_stroma_nsclc_200_rad.csv",mode = 'a') as f:
            slide_stats.to_csv(f, header=f.tell()==0,index = False)
        single_slide_neighbors(df, i, cell_types,cell_neighbors)
csv_run = create_neighbor_csv(cell_types,cell_neighbors)

 29%|██▉       | 120/414 [5:57:24<8:01:16, 98.22s/it] 

In [11]:
# in 7.5 hours this made it to 469 unique images out of 487
## this time in 6.5 it made it through all

### I might actually want to have those cd56+ patients in there too, could be interesting to analyze

In [18]:
test_df = pd.read_csv("/scratch/gh8sj/brown_nsclc/data/neighbor_data/tumor_stroma/tumor_stroma_nsclc_200_rad.csv")
test_df

,Slide Index,ImageLocation,XMin,XMax,YMin,YMax,MHCI,CD56,CD8,CD3,...,PanCyto+CD56- Neighbors,PanCyto+CD56+ Neighbors,CD3+CD8+IFNy+ Neighbors,CD3+CD8+IFNy- Neighbors,CD3+CD8-IFNy+ Neighbors,CD3+CD8-IFNy- Neighbors,CD56+CD3-IFNy+ Neighbors,CD56+CD3-IFNy- Neighbors,CD56+CD3+IFNy+ Neighbors,CD56+CD3+IFNy- Neighbors
0,0,E:\BROWN RESCAN 11.8.19\Rescan Images 11.11.19...,2002.47344,2008.90272,420.37600,429.77264,0.0,0.0,0.0,0.0,...,179.0,1.0,0.0,17.0,0.0,54.0,0.0,1.0,0.0,0.0
1,1,E:\BROWN RESCAN 11.8.19\Rescan Images 11.11.19...,2003.95712,2008.90272,445.10400,453.51152,0.0,0.0,0.0,0.0,...,179.0,1.0,0.0,17.0,0.0,61.0,0.0,1.0,0.0,0.0
2,2,E:\BROWN RESCAN 11.8.19\Rescan Images 11.11.19...,2001.48432,2008.90272,606.33056,615.23264,0.0,0.0,0.0,0.0,...,230.0,1.0,0.0,18.0,0.0,62.0,0.0,8.0,0.0,0.0
3,3,E:\BROWN RESCAN 11.8.19\Rescan Images 11.11.19...,2000.49520,2007.91360,715.13376,722.55216,0.0,0.0,1.0,1.0,...,248.0,1.0,0.0,8.0,0.0,23.0,0.0,7.0,0.0,0.0
4,4,E:\BROWN RESCAN 11.8.19\Rescan Images 11.11.19...,1988.62576,1996.53872,72.20576,80.11872,0.0,0.0,1.0,0.0,...,96.0,0.0,0.0,4.0,0.0,9.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4212770,9992,E:\Brown 4.25.19\Unmixed Images 4.29.19\MAD17-...,467.85376,476.75584,1483.68000,1491.59296,0.0,0.0,0.0,0.0,...,349.0,0.0,1.0,7.0,0.0,84.0,0.0,0.0,0.0,0.0
4212771,9993,E:\Brown 4.25.19\Unmixed Images 4.29.19\MAD17-...,458.95168,465.38096,1485.65824,1490.10928,1.0,0.0,0.0,0.0,...,342.0,0.0,1.0,6.0,0.0,84.0,0.0,0.0,0.0,0.0
4212772,9994,E:\Brown 4.25.19\Unmixed Images 4.29.19\MAD17-...,451.53328,458.95168,1490.60384,1497.52768,1.0,0.0,0.0,0.0,...,326.0,0.0,1.0,6.0,0.0,81.0,0.0,0.0,0.0,0.0
4212773,9995,E:\Brown 4.25.19\Unmixed Images 4.29.19\MAD17-...,548.96160,555.39088,1491.59296,1498.02224,1.0,0.0,0.0,0.0,...,296.0,0.0,0.0,4.0,0.0,66.0,0.0,0.0,0.0,0.0
